In [1]:
import numpy as np
import h5py
import netCDF4 as nc
import oceanDAS as odas
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import date, datetime, timezone
from scipy import signal

%matplotlib inline
%cd -q '/Users/hglover/Library/CloudStorage/Box-Box/FiberExperiments/DAScomparison_paper/'

plt.rc('font', size=12) 

In [ ]:
def calc_Dspec(das,dasTime,t1,C,C_frq):
    fs = 1/(dasTime[1]-dasTime[0])
    das = das[(dasTime>=t1) & (dasTime<(t1+60*60))]
    frq,psd_das = signal.welch(das,fs=fs,window='hann',nperseg=fs*60*5,detrend=False)

    # correct from PSD of strain to PSD of pressure
    C = np.interp(frq,C_frq,C)
    psd_dasp = C*psd_das
    
    return frq, psd_das, psd_dasp

def calc_Pspec(press,presTime,t1):
    fs = 1/(presTime[1]-presTime[0])
    press = press[(presTime>=t1) & (presTime<(t1+60*60))]
    frq,psd = signal.welch(press,fs=fs,window='hann',nperseg=fs*60*5,detrend=False)

    return frq, psd

def calc_nrmse(ctrl,pred):
    nrmse = np.sqrt(np.mean((ctrl-pred)**2))/np.mean(ctrl)
    return nrmse

In [ ]:
# Duck
pname = 'DuckNC_Glover/'

t1 = datetime(2021, 11, 12, 0, 0, 0,tzinfo=timezone.utc).timestamp()
# load pressure
with np.load(pname+'FRF-ocean_waveTs_awac-11m_202111.npz') as data:
    press = data['p']
    presTime = data['t']
frq_p, psd_p = calc_Pspec(press,presTime,t1)

# load strain
f = h5py.File(pname + 'ChNo436_Nov2021fromdown.hdf5', 'r') 
dasTime = f['Acquisition']['RawDataTime'][:]
das = f['Acquisition']['RawData'][:] # unit of E

# load correction factor
with np.load(pname+'empCalNov_all.npz') as data:
    C_frq = data['x']
    C = data['y'][381,:]
with np.load(pname+'empCalNov_all_4awac.npz') as data:
    C2 = data['y'][381,:]
frq_d, psd_d, psd_dp = calc_Dspec(das,dasTime,t1,C,C_frq)

# load other correction factor:
frq_d2, _, psd_dp2 = calc_Dspec(das,dasTime,t1,C2,C_frq)


# make the figure
plt.ioff()
fig,ax = plt.subplots(2,1,figsize=(10,5))
ax = ax.ravel()

# plot example with close calibration point
ax[0].plot(frq_d,psd_d,'b',linewidth=2)
ax2 = ax[0].twinx()
ax2.plot(frq_d,psd_dp,c = 'tab:gray',linewidth=2,linestyle='--',label='DAS-Nearest')
ax2.plot(frq_p,psd_p,c = 'k',linewidth=2.5,label='pressure')
ax2.plot(frq_d2,psd_dp2,c = 'r',linewidth=2,linestyle=':',label='DAS-Farthest')
ax2.set_ylim([0,0.5])
ax2.legend()
ax[0].set_xscale('log')
ax[0].set_xlim([0.05,0.5])
ax[0].set_ylim([0,5e-10])

# # plot the example with far calibration point
# ax[1].plot(frq_d,psd_d,'r--',linewidth=2.5)
# ax2 = ax[1].twinx()
# ax2.plot(frq_d,psd_dp2,c = 'tab:gray',linewidth=2.5,linestyle=':')
# ax2.plot(frq_p,psd_p,c = 'k',linewidth=2.5)
# ax2.set_ylim([0,0.5])

In [3]:
# KNO
pname = 'KNO_Glover/'

t1 = datetime(2023, 1, 17, 4, 0, 0,tzinfo=timezone.utc).timestamp()
# load pressure
with np.load(pname + 'rbr_pres_temp_all.npz') as data:
    press = data['p'][:,5] #5 is nearest ch124, h=11.5 m
    presTime = data['tvec']
# calculate spectra for pressure
frq_p, psd_p = calc_Pspec(press,presTime,t1)

# load strain data
chno = 124
f = h5py.File(pname+'downsampFilt_F1_01-17-2023.hdf5', 'r')
das = f['Acquisition']['RawData'][:,chno]
dasTime = f['Acquisition']['RawDataTime'][:]

# load correction factor for chno = 124, Fiber 1,
with np.load(pname+'empCal_all_F1.npz') as data:
    C_frq = data['frq']
    C = data['sf'][chno-40,:].flatten()

# load other correction factor:
with np.load(pname+'empCal_ch124_p1.npz') as data:
    C2_frq = data['f']
    C2 = data['sf']

    

# calculate spectra for corrected (pressure) and uncorrected (strain)
frq_d, psd_d, psd_dp = calc_Dspec(das,dasTime,t1,C,C_frq)
frq_d2, _, psd_dp2 = calc_Dspec(das,dasTime,t1,C2,C2_frq)
psd_dp2 += 0.005

# plot example with close calibration point
ax[1].plot(frq_d,psd_d,'b',linewidth=2)
ax2 = ax[1].twinx()
ax2.plot(frq_d,psd_dp,c = 'tab:gray',linewidth=2,linestyle='--')
ax2.plot(frq_p,psd_p,c = 'k',linewidth=2.5)
ax2.plot(frq_d2,psd_dp2,c = 'r',linewidth=2,linestyle=':')
ax2.set_ylim([0,0.3])
ax[1].set_xscale('log')
ax[1].set_xlim([0.05,0.5])
ax[1].set_ylim([0,1e-12])
# # plot the example with far calibration point
# ax[3].plot(frq_d,psd_d,'r--',linewidth=2.5)
# ax2 = ax[3].twinx()
# ax2.plot(frq_p,psd_p,c = 'k',linewidth=2.5)
# ax2.plot(frq_d2,psd_dp2,c = 'r',linewidth=2,linestyle=':')
# ax2.set_ylim([0,0.3])


plt.ion()
fig

fig.savefig('CorrectionDistanceImpacts.svg')

(0.0, 5e-10)

In [4]:
# # Oliktokdd
# pname = 'OliktokPoint_Smith/'

# t1 = datetime(2022, 8, 18, 23, 0, 0,tzinfo=timezone.utc).timestamp()

# # load swift and convert to bed pressure 
# d = nc.Dataset(pname+'SWIFT18_OliktokPt_Aug2022_reprocessedIMU_1Hzspectra_convertedTe.nc')
# tvec_swft = d['time'][:] + datetime.strptime('2022-08-13T22:00:00','%Y-%m-%dT%H:%M:%S').timestamp()
# idx = (tvec_swft>t1) & (tvec_swft<t1+60*60)
# frq_p = d['freq'][:]
# psd_p = d['energy'][idx,:].flatten() # energy(time, freq)
# h = 12
# # calculate depth attenuation
# L,_ = odas.dispersion(h,1/frq_p)
# attenuation = np.exp((h/L))**2 # square for energy
# attenuation[attenuation>150] = np.nan; # cut it off when correction too big,don't amplify noise
# psd_p = psd_p/attenuation

# # load strain
# with np.load(pname+'compiled7960strain.npz') as f:
#     das = f['e']
#     dasTime = f['t']

# # load correction factor
# # with np.load(pname+'straincorrfac7960.npz') as data:
# #     C_frq = data['f']
# #     C = data['c']
# fname = 'SpectralCorrectionFactors_1Hz/SpectralCorrectionFactor_channel7960_1Hz.csv'
# f = np.genfromtxt(pname+fname, delimiter=',',skip_header=1)
# C_frq = f[:,1]
# C = 1 / f[:,2]
# C[C_frq>0.4]=np.nan
        
# C2 = C
# # corrected vals:
# frq_d, psd_d, psd_dp = calc_Dspec(das,dasTime,t1,C,C_frq)
# _, _, psd_dp2 = calc_Dspec(das,dasTime,t1,C2,C_frq)


# ax[2].plot(frq_d,psd_d,'r--',linewidth=2.5)
# ax2 = ax[2].twinx()
# ax2.plot(frq_d,psd_dp,c = 'tab:gray',linewidth=2.5,linestyle=':')
# ax2.plot(frq_p,psd_p,c = 'k',linewidth=2.5)
# ax2.set_ylim([0,0.001])

# ax[3].plot(frq_d,psd_d,'r--',linewidth=2.5)
# ax2 = ax[3].twinx()
# ax2.plot(frq_d,psd_dp2,c = 'tab:gray',linewidth=2.5,linestyle=':')
# ax2.plot(frq_p,psd_p,c = 'k',linewidth=2.5)



In [7]:
# for jj in range(2):
#     ax[jj].set_xscale('log')
#     ax[jj].set_xlim([0.05,0.35])
# ax[0].set_ylim([0,5e-10])
# ax[1].set_ylim([0,1e-12])


# plt.ion()
# fig